In [1]:
# Cell to hide - just an option and import other functions

import pandas as pd
pd.set_option('display.max_columns', 27)

%run 1-functions.ipynb

__TODO: the whole thing__

More ideas
+ Try to predict with something that does not make sense (example: using ID attribute as a number)
+ or using Length as a category
+ mixing regression and classification

In [2]:
import pandas as pd

static_data = pd.read_csv('./static_data.csv')

In [16]:
# Prediction of Width from Length or TripID

from sklearn.metrics import mean_absolute_error

x = ['Length']
y = ['Width']
pred, ytest = knn_regression(static_data, x, y)
print('MAE with Length: ' + str(mean_absolute_error(pred, ytest)))

x = ['Length', 'TripID']
y = ['Width']
pred, ytest = knn_regression(static_data, x, y)
print('MAE with Length and TripID: ' + str(mean_absolute_error(pred, ytest)))

x = ['TripID']
y = ['Width']
pred, ytest = knn_regression(static_data, x, y)
print('MAE with TripID: ' + str(mean_absolute_error(pred, ytest)))

print('Baseline: mean = ' + str(static_data['Width'].mean()))

MAE with Length: 2.73451052631579
MAE with Length and TripID: 3.8834960526315796
MAE with TripID: 7.796980263157895
Baseline: mean = 13.10450054884743


In [17]:
# Prediction of Width from Length: comparison regression and classification

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

x = ['Length']
y = ['Width']

df = static_data.copy()
df = df.dropna()

pred, ytest = knn_regression(df, x, y)

print('MAE with regression: ' + str(mean_absolute_error(pred, ytest)))

df2 = static_data.copy()
df2 = df2.dropna()
df2['Width'] = df2['Width'].astype('int')
df2['Width'] = df2['Width'].astype('category')

pred, ytest = knn_classification(df2, x, y)

print('Accuracy with classification: ' + str(accuracy_score(pred, ytest)))

MAE with regression: 2.0717238095238097
Accuracy with classification: 0.38095238095238093


C:\Users\Anna\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
